In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2020-12-22 14:58:24,668 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-12-22 14:58:24,669 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [3]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2020-12-22 14:59:01,889 : INFO : collecting document frequencies
2020-12-22 14:59:01,891 : INFO : PROGRESS: processing document #0
2020-12-22 14:59:01,897 : INFO : calculating IDF weights for 9 documents and 12 features (28 matrix non-zeros)


In [4]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [5]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [6]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2020-12-22 15:15:10,914 : INFO : using serial LSI version on this node
2020-12-22 15:15:10,915 : INFO : updating model with new documents
2020-12-22 15:15:10,918 : INFO : preparing a new chunk of documents
2020-12-22 15:15:10,922 : INFO : using 100 extra samples and 2 power iterations
2020-12-22 15:15:10,924 : INFO : 1st phase: constructing (12, 102) action matrix
2020-12-22 15:15:10,926 : INFO : orthonormalizing (12, 102) action matrix
2020-12-22 15:15:10,934 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2020-12-22 15:15:10,937 : INFO : computing the final decomposition
2020-12-22 15:15:10,938 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2020-12-22 15:15:10,940 : INFO : processed documents up to #9
2020-12-22 15:15:10,943 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2020-12-22 15:15:10,944 : INFO : topic #

In [7]:
lsi_model.print_topics(2)

2020-12-22 15:15:20,206 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2020-12-22 15:15:20,208 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [8]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090225), (1, -0.5200703306361849)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142385), (1, -0.7609563167700045)] A survey of user opinion of computer system response time
[(0, 0.08992639972446312), (1, -0.7241860626752512)] The EPS user interface management system
[(0, 0.0758584765217807), (1, -0.6320551586003432)] System and human system engineering testing of EPS
[(0, 0.10150299184980027), (1, -0.5737308483002954)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378311), (1, 0.16115180214025654)] The generation of random binary unordered trees
[(0, 0.8774787673119834), (1, 0.16758906864659254)] The intersection graph of paths in trees
[(0, 0.9098624686818583), (1, 0.14086553628718876)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569287), (1, -0.053929075663894266)] Graph minors A survey
